In [4]:
# Importação de bibliotecas
import pandas as pd

from pendulum import now

### Bronze - Prata

In [37]:
# Variáveis utilizadas
fonte = 'sidra'
dataset_name = 'risco_erosao_lavouras_perenes'
variavel = 'area_colhida'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

ultimo_ano = now().year - 1

In [14]:
df = (
    pd.read_json('https://apisidra.ibge.gov.br/values/t/1613/n3/all/u/y/v/216/p/last%2024/c82/0,2717,2718,2719,2720,2721,2722,2723,2724,2725,2726,2727,2728,2729,2730,2731,2732,2733,2734,2735,2736,2737,2738,2739,2740,2741,2742,2743,2744,2745,2746,2747,2748,31619,31620,45981,90001')
    .drop(0, axis = 0)
        .filter(['D1C', 'D3C', 'D4C', 'V'])
        .assign(**{
            'valor': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'geocodigo': lambda df: df['D1C'].astype(int),
            'data': lambda df: pd.to_datetime(df['D3C'], format='%Y'),
            'cultura': lambda df: df['D4C'].astype(int)
            })
        .drop(['D1C', 'D3C', 'D4C', 'V'], axis='columns')
        .dropna(subset='valor')
        .query('cultura != 0')
        .set_index(['geocodigo', 'data', 'cultura'])
)

### Ouro

In [21]:
df_cafe = (
    df
    .reset_index()
    .query('cultura == 2723')
    .replace({2723: 'Café - Convencional + cobertura do solo'})
    .assign(valor = lambda n: round(n.valor / 1000000, 2))
    .reset_index(drop=True)
)

In [49]:
df_frutas = (
    df
    .reset_index()
    .query('cultura != 2723')
    .groupby(['geocodigo', 'data'])
    .agg({'valor':'sum'})
    .reset_index()
    .assign(**{
        'Frutíferas - Sem cobertura do solo (solo nu)': lambda n: round(n.valor / 1000000, 2),
        'Frutíferas - Com proteção à erosão': lambda n: round(n.valor / 1000000, 2)
        })
    .drop(columns='valor')
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
)

In [53]:
df_final = (
    pd.concat([
        df_cafe,
        df_frutas
    ])
    .sort_values(['geocodigo', 'data'])
    .reset_index(drop=True)
)

In [54]:
df_final

,geocodigo,data,cultura,valor
0,11,2000-01-01,Café - Convencional + cobertura do solo,0.2
1,11,2000-01-01,Frutíferas - Sem cobertura do solo (solo nu),0.05
2,11,2000-01-01,Frutíferas - Com proteção à erosão,0.05
3,11,2001-01-01,Café - Convencional + cobertura do solo,0.22
4,11,2001-01-01,Frutíferas - Sem cobertura do solo (solo nu),0.04
...,...,...,...,...
1709,53,2022-01-01,Frutíferas - Sem cobertura do solo (solo nu),0.0
1710,53,2022-01-01,Frutíferas - Com proteção à erosão,0.0
1711,53,2023-01-01,Café - Convencional + cobertura do solo,0.0
1712,53,2023-01-01,Frutíferas - Sem cobertura do solo (solo nu),0.0


In [55]:
df_ouro = (
    df_final
    .rename(columns={
        'geocodigo': 'state',
        'valor': 'value',
        'data': 'date',
        'cultura': 'label'
        })
    .assign(country = 76)
    .filter(['country', 'state', 'date', 'label', 'value'])
)

df_ouro.to_csv(f'{raiz}/ouro_{dataset_name}.csv', index=False)